<a href="https://colab.research.google.com/github/sizhky/AutoTrain/blob/segmentation/nbs/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Data and Libraries

In [1]:
%%capture
# Setup imports
try:
    import timm
except:
    !pip install -U torch-snippets typer imgaug timm fastai

In [2]:
# Setup Data
%load_ext autoreload
%autoreload 2

from torch_snippets import *

# If running locally from AutoTrain/nbs
if not os.path.exists('/content') and P().resolve().stem != 'AutoTrain':
    %cd ..

# If running from colab
if os.path.exists('/content') and not os.path.exists('/content/AutoTrain'):
    !git clone https://github.com/sizhky/AutoTrain

# change present working directory
if not P().resolve().stem == 'AutoTrain':
    %cd AutoTrain

# !git pull origin enhancement/fastai-classification
# !git checkout enhancement/fastai-classification

if not os.path.exists('imagenette2-160'):
    untar_data(URLs.IMAGENETTE_160).mv('./')


/mnt/d04d289c-d0a7-4fd4-b883-ff227b3a8695/githubs/AutoTrain


## Verify Config

In [3]:
!cat config_classification.ini

[project]
owner: sizhky
name: imagenette2
location: ${project.name}
model_directory: ${project.location}/models
type: classification
num_classes: 10

[label_studio]
base_data_dir: /tmp/data/

[label_studio_ml]
app_name: ${project.name}/my_ml_model
script_path: auto_train_classification/script_classification.py

[online_training]
save_dir: ${project.model_directory}/model.pt

[architecture]
image_size: 224
[architecture.backbone]
model: efficientnet_b0
# everytime you change the model, vector size 
# must match the model's backbone output.
vector_size: 1280
[architecture.head]
@head: custom_head
head_input_size: ${architecture.backbone.vector_size}
num_classes: ${project.num_classes}

[training]
dir: imagenette2-160
train_ratio: 0.8
[training.preprocess]
@preprocess_function: my_preprocess
image_size: ${architecture.image_size}

[training.data]
train_dir: ${training.dir}/train
validation_dir: ${training.dir}/validation

[training.scheme]
epochs: 3
freeze_epochs: 2
initial_checkpoint: ${

In [8]:
os.environ['CONFIG'] = 'config_classification.ini'
!python auto_train_classification/train.py

2021-09-12 15:44:53.050 | INFO     | __main__:train_model:32 - Training from scratch!
2021-09-12 15:45:10.372 | INFO     | __main__:find_best_learning_rate:22 - LR Plot is saved at imagenette2/find_lr_plot.png
2021-09-12 15:45:10.372 | INFO     | __main__:find_best_learning_rate:23 - Suggested LRs: 0.0006309573538601399 and 0.001737800776027143
2021-09-12 15:45:10.372 | INFO     | __main__:train_model:37 - Using lr: 0.001737800776027143
['Epoch, Train Loss, Validation Loss, Validation Accuracy, Time']
[0, 0.8093251585960388, 0.5401224493980408, 0.8287898302078247, '00:22']
[0, 0.5413949489593506, 0.3546662926673889, 0.8896815180778503, '00:26']
2021-09-12 15:45:59.883 | INFO     | __main__:train_model:47 - Saved weights of size ~18 MB to imagenette2/models/trained-model-v1/my_model.pt


---
## Validate

In [26]:
from auto_train_classification.infer import infer, learn
infer('imagenette2-160/val/n01440764/')

[(» imagenette2-160/val/n01440764/n01440764_11881.JPEG,
  'n01440764',
  tensor(0.99511)),
 (» imagenette2-160/val/n01440764/n01440764_12971.JPEG,
  'n01440764',
  tensor(0.58467)),
 (» imagenette2-160/val/n01440764/n01440764_1951.JPEG,
  'n03888257',
  tensor(0.25995)),
 (» imagenette2-160/val/n01440764/n01440764_12250.JPEG,
  'n01440764',
  tensor(0.97597))]